In [1]:
import numpy as np
import pickle

import qutip
from pymoo.core.problem import ElementwiseProblem,StarmapParallelization
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.soo.nonconvex.nelder import NelderMead

import multiprocessing


# CoupledQuantumSystems (as from your code):
from CoupledQuantumSystems.drive import DriveTerm, square_pulse_with_rise_fall
from CoupledQuantumSystems.IFQ import gfIFQ
from CoupledQuantumSystems.evo import ODEsolve_and_post_process


# -------------------------------
# Define your parameters and objects
# -------------------------------
EJ = 3
EJoverEC = 6
EJoverEL = 25
EC = EJ / EJoverEC
EL = EJ / EJoverEL

qbt = gfIFQ(EJ=EJ, EC=EC, EL=EL, flux=0, truncated_dim=20)
e_ops = [
    qutip.basis(qbt.truncated_dim, i) * qutip.basis(qbt.truncated_dim, i).dag()
    for i in range(10)
]

element = np.abs(qbt.fluxonium.matrixelement_table("n_operator", evals_count=3)[1, 2])
freq = (qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]) * 2 * np.pi

# --------------------------------------------------------------
# DEFINE THE OBJECTIVE FUNCTION
# x = (t_tot, amp, w_d)
# We want to minimize sum of infidelities for |1>->|2> and |2>->|1>.
# --------------------------------------------------------------
def fluxonium_objective(t_tot, amp, w_d,ramp):
    tlist = np.linspace(0, t_tot * (1+ramp), 100)

    initial_states = [
        qutip.basis(qbt.truncated_dim, 1),
        qutip.basis(qbt.truncated_dim, 2),
    ]

    drive_terms = [
        DriveTerm(
            driven_op=qutip.Qobj(qbt.fluxonium.n_operator(energy_esys=True)),
            pulse_shape_func=square_pulse_with_rise_fall,
            pulse_id="pi",
            pulse_shape_args={
                "w_d": w_d,    # No extra 2pi factor
                "amp": amp,    # No extra 2pi factor
                "t_square": t_tot * (1-ramp),
                "t_rise": t_tot * ramp,
            },
        )
    ]

    # Solve the dynamics for each initial state
    results = []
    for init_state in initial_states:
        res = ODEsolve_and_post_process(
            y0=init_state,
            tlist=tlist,
            static_hamiltonian=qbt.diag_hamiltonian,
            drive_terms=drive_terms,
            e_ops=e_ops,
            print_progress=False,
        )
        results.append(res)

    # Calculate final populations
    # (We want to transfer |1> -> |2> and |2> -> |1>.)
    one_minus_pop2 = abs(1 - results[0].expect[2][-1])  # from state |1> to |2>
    one_minus_pop1 = abs(1 - results[1].expect[1][-1])  # from state |2> to |1>

    return one_minus_pop2 + one_minus_pop1


# --------------------------------------------------------------
# CREATE A PYMOO PROBLEM SUBCLASS
# --------------------------------------------------------------
class FluxoniumProblem(ElementwiseProblem):
    """
    We define a single-objective problem with:
      - n_var=3: (amp, w_d,ramp)
      - n_obj=1: we want to minimize a single objective
      - n_constr=1: we add a constraint G <= 0 => t_tot>0 => -t_tot < 0
      - x_lower and x_upper define bounding box
    """

    def __init__(self,t_tot):
        self.t_tot = t_tot
        # init_amp = np.pi / element / t_tot / 3.1415 / 2
        init_wd = qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]
        amp_guess = 50/t_tot * 22

        xl = np.array([amp_guess/5, 0.003, 1e-10])
        xu = np.array([amp_guess*5, 0.02, 0.5])

        # n_constr=1 means we'll define 1 constraint
        super().__init__(
            n_var=3,
            n_obj=1,
            n_constr=0,
            xl=xl,
            xu=xu,
            elementwise_evaluation=True  # we evaluate one solution at a time
        )

    def _evaluate(self, x, out, *args, **kwargs):
        """
        x is a single solution [amp, w_d,ramp] because we use elementwise_evaluation=True
        We'll compute:
          out["F"] = fluxonium_objective(x)
          out["G"] = the constraint violation (needs to be <= 0 to be feasible)
        """
        amp, w_d,ramp = x

        # The objective:
        f = fluxonium_objective(self.t_tot, amp, w_d,ramp)

        out["F"] = f
        out["G"] = -t_tot

n_proccess = 20

# define the problem by passing the starmap interface of the thread pool
t_tot_list = np.linspace(50, 250, 21)
for t_tot in t_tot_list:
    pool = multiprocessing.Pool(n_proccess)
    runner = StarmapParallelization(pool.starmap)
    problem = FluxoniumProblem(t_tot)
    res = minimize(problem, GA(), termination=("n_gen", 200), seed=1,verbose=True)
    pool.close()
    print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

    results_dict = {
        't_tot': t_tot,
        'res': res
    }

    with open(f"pymoo_optimized_results_{t_tot}.pkl", "wb") as f:
        pickle.dump(results_dict, f)

capi_return is NULL
Call-back cb_f_in_zvode__user__routines failed.


KeyboardInterrupt: 